In [1]:
import lotr

In [2]:
conf = lotr.read_conf()
sheet_path = lotr.download_sheet(conf)
sets = lotr.get_sets(conf, sheet_path)

In [3]:
for set_data in sets:
    set_id, set_name, set_row = set_data
    print('Processing set ID {}'.format(set_id))
    lotr.backup_previous_xml(conf, set_id)
    lotr.generate_xmls(sheet_path, set_row)

Processing set ID 8a3273ca-1ccd-4e07-913b-766fcc49fe6f
Processing set ID 17b90ca1-7e3e-4891-99da-954a7833e88e


In [ ]:
# Clear XML files in the project folder
clear_folder(XML_PATH)

# Copy setEons.xml into the project
_ = shutil.copyfile(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml'),
                    os.path.join(XML_PATH, '{}.xml'.format(SET_UUID)))

In [ ]:
# Create Strange Eons project archive
with zipfile.ZipFile(PROJECT_PATH, 'w') as zip_obj:
    for root, _, filenames in os.walk(PROJECT_FOLDER):
        for filename in filenames:
            zip_obj.write(os.path.join(root, filename))

### Manual Strange Eons Part:

1. Open `setGenerator.seproject` generated above.
2. Run `Script/makeCards` script by double clicking it.  This will take the templates, fill them with values from `setEons.xml` and export results in all the required formats.
3. Once completed, close Strange Eons.

Run the cells below:

In [ ]:
# Create the folder if needed
def create_folder(folder):
    if not os.path.exists(folder):
        os.mkdir(folder)

# Delete folder images for outdated or modified cards
def clear_modified_images(folder, skip_ids):
    for _, _, filenames in os.walk(folder):
        for filename in filenames:
            if filename.split('.')[-1] in ('jpg', 'png'):
                card_id = filename[50:86]
                if card_id not in skip_ids:
                    os.remove(os.path.join(folder, filename))

        break

# Update filename found in the Strange Eons archive
def update_zip_filename(filename):
    output_filename = os.path.split(filename)[-1]
    output_filename = output_filename.encode('ascii', errors='replace').decode().replace('?', ' ')
    parts = output_filename.split('.')
    output_filename = '.'.join(parts[:-2] + [parts[-1]])
    output_filename = re.sub('-2-1(?=\.(?:jpg|png)$)', '-2', output_filename)
    return output_filename

In [ ]:
# Generate images for DB and OCTGN outputs
output_path = os.path.join(IMAGES_EONS_PATH, 'jpg300NoBleed', SET_UUID)
create_folder(output_path)
clear_modified_images(output_path, skip_ids)

with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
    filelist = [f for f in zip_obj.namelist()
                if f.startswith('{}{}'.format(IMAGES_ZIP_PATH, 'jpg300NoBleed'))
                and f.split('.')[-1] == 'jpg' and f.split('.')[-2] == SET_UUID]
    for filename in filelist:
        output_filename = update_zip_filename(filename)
        with zip_obj.open(filename) as zip_file:
            with open(os.path.join(output_path, output_filename), 'wb') as output_file:
                _ = shutil.copyfileobj(zip_file, output_file)

In [ ]:
# Generate images for PDF outputs
output_path = os.path.join(IMAGES_EONS_PATH, 'png300PDF', SET_UUID)
create_folder(output_path)
clear_modified_images(output_path, skip_ids)
clear_folder(TEMP_PATH)

with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
    filelist = [f for f in zip_obj.namelist()
                if f.startswith('{}{}'.format(IMAGES_ZIP_PATH, 'png300Bleed'))
                and f.split('.')[-1] == 'png' and f.split('.')[-2] == SET_UUID]
    for filename in filelist:
        output_filename = update_zip_filename(filename)
        if output_filename.endswith('-2.png'):
            with zip_obj.open(filename) as zip_file:
                with open(os.path.join(TEMP_PATH, output_filename), 'wb') as output_file:
                    _ = shutil.copyfileobj(zip_file, output_file)

cmd = '"{}" -i -b "(python-prepare-pdf-back-folder 1 \\"{}\\" \\"{}\\")" -b "(gimp-quit 0)"'.format(
    conf['gimp_console_path'],
    TEMP_PATH.replace('\\', '\\\\'),
    output_path.replace('\\', '\\\\'))
print(subprocess.run(cmd, capture_output=True, shell=True))

clear_folder(TEMP_PATH)

with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
    filelist = [f for f in zip_obj.namelist()
                if f.startswith('{}{}'.format(IMAGES_ZIP_PATH, 'png300NoBleed'))
                and f.split('.')[-1] == 'png' and f.split('.')[-2] == SET_UUID]
    for filename in filelist:
        output_filename = update_zip_filename(filename)
        if output_filename.endswith('-1.png'):
            with zip_obj.open(filename) as zip_file:
                with open(os.path.join(TEMP_PATH, output_filename), 'wb') as output_file:
                    _ = shutil.copyfileobj(zip_file, output_file)

cmd = '"{}" -i -b "(python-prepare-pdf-front-folder 1 \\"{}\\" \\"{}\\")" -b "(gimp-quit 0)"'.format(
    conf['gimp_console_path'],
    TEMP_PATH.replace('\\', '\\\\'),
    output_path.replace('\\', '\\\\'))
print(subprocess.run(cmd, capture_output=True, shell=True))

In [ ]:
# Generate images for MakePlayingCards outputs
output_path = os.path.join(IMAGES_EONS_PATH, 'png800BleedMPC', SET_UUID)
create_folder(output_path)
clear_modified_images(output_path, skip_ids)
clear_folder(TEMP_PATH)

with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
    filelist = [f for f in zip_obj.namelist()
                if f.startswith('{}{}'.format(IMAGES_ZIP_PATH, 'png800Bleed'))
                and f.split('.')[-1] == 'png' and f.split('.')[-2] == SET_UUID]
    for filename in filelist:
        output_filename = update_zip_filename(filename)
        with zip_obj.open(filename) as zip_file:
            with open(os.path.join(TEMP_PATH, output_filename), 'wb') as output_file:
                _ = shutil.copyfileobj(zip_file, output_file)

cmd = '"{}" -i -b "(python-prepare-makeplayingcards-folder 1 \\"{}\\" \\"{}\\")" -b "(gimp-quit 0)"'.format(
    conf['gimp_console_path'],
    TEMP_PATH.replace('\\', '\\\\'),
    output_path.replace('\\', '\\\\'))
print(subprocess.run(cmd, capture_output=True, shell=True))

In [ ]:
clear_folder(TEMP_PATH)

In [ ]:
# Generate DB outputs
input_path = os.path.join(IMAGES_EONS_PATH, 'jpg300NoBleed', SET_UUID)
output_path = os.path.join(OUTPUT_DB_PATH, set_name)
create_folder(output_path)
clear_folder(output_path)

known_filenames = set()
for _, _, filenames in os.walk(input_path):
    for filename in filenames:
        if filename.split('.')[-1] != 'jpg':
            continue

        db_filename = '{}-{}{}{}'.format(filename[:3], re.sub('-+$', '',filename[8:50]),
                                         re.sub('-1$', '', filename[86:88]), filename[88:])
        if db_filename not in known_filenames:
            known_filenames.add(db_filename)                
            _ = shutil.copyfile(os.path.join(input_path, filename),
                                os.path.join(output_path, db_filename))

    break

In [ ]:
# Generate OCTGN outputs
input_path = os.path.join(IMAGES_EONS_PATH, 'jpg300NoBleed', SET_UUID)
output_path = os.path.join(OUTPUT_OCTGN_PATH, SET_UUID)
create_folder(output_path)

known_filenames = set()
pack_path = os.path.join(output_path, 'imagePack_{}.o8c'.format(SET_UUID))
with zipfile.ZipFile(pack_path, 'w', zipfile.ZIP_DEFLATED) as zip_obj:
    for _, _, filenames in os.walk(input_path):
        for filename in filenames:
            if filename.split('.')[-1] != 'jpg':
                continue

            parts = filename.split('.')
            parts[0] = re.sub('-1$', '', re.sub('-2$', '.B', parts[0]))
            octgn_filename = '.'.join(parts)[50:]
            if octgn_filename not in known_filenames:
                known_filenames.add(octgn_filename)
                zip_obj.write(os.path.join(input_path, filename),
                             '{}/{}/Cards/{}'.format(OCTGN_ZIP_PATH, SET_UUID, octgn_filename))

        break

In [ ]:
# Generate PDF outputs
input_path = os.path.join(IMAGES_EONS_PATH, 'png300PDF', SET_UUID)
output_path = os.path.join(OUTPUT_PDF_PATH, set_name)
create_folder(output_path)

pdf_images = {'player': [],
              'encounter': [],
              'custom': []}
for _, _, filenames in os.walk(input_path):
    for filename in filenames:
        parts = filename.split('-')
        if parts[-1] != '1.png':
            continue

        back_filepath = os.path.join(input_path, '{}-2.png'.format('-'.join(parts[:-1])))
        if os.path.exists(back_filepath):
            back_type = 'custom'
        else:
            if parts[2] == 'p':
                back_type = 'player'
                back_filepath = os.path.join(IMAGES_BACK_PATH, 'playerBackOfficial.png')
            elif parts[2] == 'e':
                back_type = 'encounter'
                back_filepath = os.path.join(IMAGES_BACK_PATH, 'encounterBackOfficial.png')
            else:
                print('Missing card back for {}, removing the file from PDF'.format(filename))
                continue

        copies = 3 if parts[1] == 'p' else 1
        for _ in range(copies):
            pdf_images[back_type].append((os.path.join(input_path, filename), back_filepath))
                
    break

cards_on_page = 6
pages_raw = []
for key in pdf_images:
    pages_raw.extend([(pdf_images[key][i * cards_on_page:(i + 1) * cards_on_page] +
                       [None] * cards_on_page)[:cards_on_page]
                      for i in range(math.ceil(len(pdf_images[key]) / cards_on_page))])

pages = []
for page in pages_raw:
    front_page = [i and i[0] or None for i in page]
    back_page = [i and i[1] or None for i in page]
    back_page = [back_page[2], back_page[1], back_page[0],
                 back_page[5], back_page[4], back_page[3]]
    pages.extend([front_page, back_page])

formats = {'A4': A4, 'letter': letter}
card_width = 2.75 * inch
card_height = 3.75 * inch

for page_format in formats:
    canvas = Canvas(os.path.join(output_path, '{}.pdf'.format(page_format)),
                    pagesize=landscape(formats[page_format]))
    width, height = landscape(formats[page_format])
    width_margin = (width - 3 * card_width) / 2
    height_margin = (height - 2 * card_height) / 2
    for page in pages:
        for i in range(len(page)):
            if page[i]:
                width_pos = (width_margin + i * card_width
                             if i < cards_on_page / 2
                             else width_margin + (i - cards_on_page / 2) * card_width)
                height_pos = height_margin + card_height if i < cards_on_page / 2 else height_margin
                canvas.drawImage(page[i], width_pos, height_pos,
                                 card_width, card_height, anchor='sw')

        canvas.showPage()

    canvas.save()

In [ ]:
# Generate MakePlayingCards outputs
input_path = os.path.join(IMAGES_EONS_PATH, 'png800BleedMPC', SET_UUID)
output_path = os.path.join(OUTPUT_MPC_PATH, set_name)
create_folder(output_path)

with zipfile.ZipFile(os.path.join(output_path, '{}.zip'.format(set_name)), 'w') as zip_obj:
    with py7zr.SevenZipFile(os.path.join(output_path, '{}.7z'.format(set_name)), 'w') as sevenz_obj:
        for _, _, filenames in os.walk(input_path):
            for filename in filenames:
                parts = filename.split('-')
                if parts[-1] != '1.png':
                    continue

                back_filepath = os.path.join(input_path, '{}-2.png'.format('-'.join(parts[:-1])))
                if not os.path.exists(back_filepath):
                    if parts[2] == 'p':
                        back_filepath = os.path.join(IMAGES_BACK_PATH, 'playerBackUnofficialMPC.png')
                    elif parts[2] == 'e':
                        back_filepath = os.path.join(IMAGES_BACK_PATH, 'encounterBackUnofficialMPC.png')
                    else:
                        print('Missing card back for {}, removing the file from MakePlayingCards archive'
                              .format(filename))
                        continue

                if parts[1] == 'p':
                    for i in range(3):
                        parts[1] = str(i + 1)
                        front_zippath = re.sub('-(?:e|p)-', '-', re.sub('-+', '-', re.sub('.{36}(?=-1\.png)', '',
                                                              'front/{}'.format('-'.join(parts)))))
                        back_zippath = re.sub('-(?:e|p)-', '-', re.sub('-+', '-', re.sub('.{36}(?=-2\.png)', '',
                                                             'back/{}'.format('{}-2.png'.format('-'.join(parts[:-1]))))))
                        zip_obj.write(os.path.join(input_path, filename), front_zippath)
                        zip_obj.write(back_filepath, back_zippath)
                        sevenz_obj.write(os.path.join(input_path, filename), front_zippath)
                        sevenz_obj.write(back_filepath, back_zippath)
                else:
                    front_zippath = re.sub('-(?:e|p)-', '-', re.sub('-+', '-', re.sub('.{36}(?=-1\.png)', '',
                                                          'front/{}'.format('-'.join(parts)))))
                    back_zippath = re.sub('-(?:e|p)-', '-', re.sub('-+', '-', re.sub('.{36}(?=-2\.png)', '',
                                                         'back/{}'.format('{}-2.png'.format('-'.join(parts[:-1]))))))
                    zip_obj.write(os.path.join(input_path, filename), front_zippath)
                    zip_obj.write(back_filepath, back_zippath)
                    sevenz_obj.write(os.path.join(input_path, filename), front_zippath)
                    sevenz_obj.write(back_filepath, back_zippath)

            break

Now there should be the following outputs:

1. `Output/DB/<set name>/`: 300 dpi JPG images for general purposes.
2. `Output/MakePlayingCards/<set name>`: `zip` and `7z` archives of 800 dpi PNG images
    to be printed on MakePlayingCards.com.
3. `Output/OCTGN/<octgnid>/:` `set.xml` and `o8c` image pack (300 dpi JPG) for OCTGN.
    Add the latter using the "Add Image Packs" button from within OCTGN.
4. `Output/PDF/<set name>/`: PDF files in `A4` and `letter` format for home printing.

In [ ]:
# Copy set folder to OCTGN directory
#from distutils.dir_util import copy_tree

#dirOCTGN = os.path.expanduser('~') + '\\Documents\\OCTGN\\'
#if not os.path.isdir(dirOCTGN):
#    print('Invalid path!')

#setspath = dirOCTGN + '\\GameDatabase\\a21af4e8-be4b-4cda-a6b6-534f9717391f\\Sets\\'

#if os.path.isdir(dirOCTGN):
#    if os.path.isdir(os.path.join(OUTPUT_OCTGN_PATH, SET_UUID)):
#        copy_tree(os.path.join(OUTPUT_OCTGN_PATH, SET_UUID), os.path.join(setspath, SET_UUID))
#        print('Copied successfully.')
#    else:
#        print('No folder found to copy.')
#else:
#    print('OCTGN Sets folder not found.')